## Import

In [14]:
import pandas as pd
import folium

### 관련 데이터 삽입

In [15]:
path = './data/지도/df_fin.csv'
df = pd.read_csv(path)
df

,Unnamed: 0,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,gu,주소,위도,경도,cnt
0,0,금곡도서관,부산 북구 효열로203번길 34-0,문화시설,문화생활시설기타,Buk-gu,부산 북구 효열로203번길 34-0,35.263010,129.017510,65
1,1,수덕사,부산 북구 상학산복길 107-0,역사관광지,절,Buk-gu,부산 북구 상학산복길 107-0,35.216131,129.039053,65
2,2,구포대리마을커뮤니티센터,부산 북구 백양대로1172번길 18-0,문화시설,문화생활시설기타,Buk-gu,부산 북구 백양대로1172번길 18-0,35.206401,129.005594,65
3,3,만덕사,부산 북구 구만덕로 63-13,역사관광지,절,Buk-gu,부산 북구 구만덕로 63-13,35.214870,129.041720,65
4,4,만덕사,부산 북구 덕천로386번길 47-18,역사관광지,절,Buk-gu,부산 북구 덕천로386번길 47-18,35.208188,129.044809,65
...,...,...,...,...,...,...,...,...,...,...
1377,1377,연화사,부산 연제구 경기장로 25-25,역사관광지,절,Yeonje-gu,부산 연제구,35.176500,129.079300,55
1378,1378,월륜사,부산 연제구 고분로 284-0,역사관광지,절,Yeonje-gu,부산 연제구 고분로 284,35.185649,129.087249,55
1379,1379,밤골작은도서관,부산 연제구 금련로24번길 33-0,문화시설,문화생활시설기타,Yeonje-gu,부산 연제구 금련로24번길 33-0,35.171950,129.095164,55
1380,1380,공덕사,부산 연제구 황령산로 497-1,역사관광지,절,Yeonje-gu,부산 연제구 황령산로 497-1,35.158915,129.082773,55


## 지도 데이터 로드

부산시 행정구역 지도(geoJson)

In [16]:
# 지도 데이터 로드
# 부산 구군 행정구역이 나눠져 있음
file = open('./data/지도/map4.geojson', 'r', encoding='utf-8')
map = file.read()

## 로드한 지도 데이터 투사할 기본 지도 설정

In [17]:
# 로드한 지도 데이터를 투사할 기본 지도 설정
m = folium.Map(
    location = [35.210419382538085, 129.08972725789795] # 부산시 중심
    , zoom_start = 11 #확대 범위
)
m

## 구간 분할

In [18]:
# 지도에 색을 입힐 때 세세하게 하기 위해 구간 분할
# quantile() 안에는 0~1 사이 값이 들어감
myscale = (df['cnt'].quantile((0,0.2,0.3,0.4,0.6,0.7,0.8,0.9,0.98,1))).tolist()

# 지도 표현

### 부산시 내 구군별 정량적인 문화시설의 수를 히트맵으로 지도에 시각화

In [19]:
# 지도 그려 투사
choropleth = folium.Choropleth(
    geo_data = map, # 사용할 지도 데이터
    data = df, # 분석 대상 데이터 프레임
    columns = ('gu', 'cnt'), # 지도에 사용할 컬럼 지정
    key_on = "feature.properties.gu", # 색을 다르게 할 기준 데이터 지정
    fill_color = 'YlOrRd', # 색 설정
    fill_opacity = 9, # 투명도 설정
    threshold_scale = myscale # 색 입힐 구간 설정 / 구간을 세분화할수록 색 단계 많아짐
).add_to(m) # 지도에 투사하라

# 마우스를 갖다대면 정보가 나오게 하는 기능
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name','count', 'pop'], aliases = ['구군명', '문화시설수', '인구'], labels= True)
)

# 지도 확인
m
#m.save("./ddd.html")

## 부산시 내 구군별 인구 대비 문화시설 수를 지도에 시각화

#### 데이터 삽입

In [20]:
df_co = pd.read_csv("./save/인구대비.csv")
df_co

,구군명,gu,소계,인구(천명),인구 대비 문화시설비
0,강서구,Gangseo-gu,54,137.957,0.391426
1,금정구,Geumjeong-gu,133,232.666,0.571635
2,기장군,Gijang-gun,153,174.545,0.876565
3,남구,Nam-gu,85,267.731,0.317483
4,동구,Dong-gu,80,88.901,0.899877
5,동래구,Dongnae-gu,98,270.745,0.361964
6,부산진구,Busanjin-gu,127,359.886,0.352890
7,북구,Buk-gu,65,283.952,0.228912
8,사상구,Sasang-gu,56,212.586,0.263423
9,사하구,Saha-gu,68,311.757,0.218119


In [21]:
# 지도 데이터 로드
# 부산 구군 행정구역이 나눠져 있음
file = open('./data/지도/map_round.geojson', 'r', encoding='utf-8')
map2 = file.read()

In [22]:
# 지도에 색을 입힐 때 세세하게 하기 위해 구간 분할
# quantile() 안에는 0~1 사이 값이 들어감
myscale2 = (df_co['인구 대비 문화시설비'].quantile((0,0.2,0.3,0.4,0.6,0.7,0.8,0.9,0.98,1))).tolist()

In [23]:
# 지도 그려 투사
choropleth = folium.Choropleth(
    geo_data = map2, # 사용할 지도 데이터
    data = df_co, # 분석 대상 데이터 프레임
    columns = ('gu', '인구 대비 문화시설비'), # 지도에 사용할 컬럼 지정
    key_on = "feature.properties.gu", # 색을 다르게 할 기준 데이터 지정
    fill_color = 'YlOrRd', # 색 설정
    fill_opacity = 9, # 투명도 설정
    threshold_scale = myscale2 # 색 입힐 구간 설정 / 구간을 세분화할수록 색 단계 많아짐
).add_to(m) # 지도에 투사하라

# 마우스를 갖다대면 정보가 나오게 하는 기능
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name','ratio', 'pop'], aliases = ['구군명', '인구 대비 문화시설비', '인구(천명)'], labels= True)
)

# 지도 확인
m
m.save("./m2.html")

In [13]:
m.save("./ddd.html")